In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import requests

C:\Users\yuppa\AppData\Local\Temp/ipykernel_9228/949280284.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\yuppa\AppData\Local\Temp/ipykernel_9228/949280284.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
#download the CSV file

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

def download(url, filename):
    response = requests.get(url)
    if response.status_code==200:
        with open(filename, 'wb') as f:
            f.write(response.content)
            

download(URL,'spacex_csv_file.csv')
spacex_df=pd.read_csv('spacex_csv_file.csv')


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [36]:
spacex_df.columns

Index(['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site',
       'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome',
       'class', 'Lat', 'Long'],
      dtype='object')

In [12]:
# create a dash application
app = dash.Dash(__name__)

# build dash layout
app.layout = html.Div(
                        children= 
                                #Dashboard Heading
                                [html.H1('SpaceX Launch Records Dashboard',
                                       style={'textalign':'center',
                                              'color': '#503D36',
                                              'font-size': 20}),
                                html.Div([
                                        html.Div([
                                        #Launch Site dropdown menu
                                            dcc.Dropdown(id='site-dropdown',
                                                    options=[{'label':'All Sites', 'value':'All'},
                                                              {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                              {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                              {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                              {'label':'VAFB SLC-4', 'value':'VAFB SLC-4E'}],
                                                    value='All',
                                                    placeholder='Select a launch site here',
                                                    searchable=True
                                                    )
                                             ]),
                                        #Success Pie Chart
                                        html.Div(dcc.Graph(id='success-pie-chart'))
                                        ]),
                                 
                                html.Div([
                                        #Payload Range Slider heading
                                        html.Div(html.H2('Payload Range (kg)',
                                                           style={'textalign':'center',
                                                                  'color': '#503D36',
                                                                  'font-size': 20}
                                                        )
                                                ),
                                        #Payload Mass range slider
                                        html.Div(dcc.RangeSlider(id = 'payload-slider',
                                                             min=0, max=10000, step=1000,
                                                             marks={2500:'2500',5000:'5000',7500:'7500'},
                                                             value=[spacex_df['Payload Mass (kg)'].min(),spacex_df['Payload Mass (kg)'].max()]
                                                                )
                            
                                                ),
                                        html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                
                                        ])
    
                                ]
                    )



@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown',component_property='value'))

def get_pie_chart(entered_site):
    
    
    if entered_site == 'All':
        filtered_df = spacex_df[spacex_df['class']==1]
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='All Launch Sites')
        return fig

    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df,values='class count',names='class',title=f"Launch Success Rate for site: {entered_site}")        #fig2 = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version')
        return fig
    

@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])

def get_scatter_chart(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    if entered_site=='All':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version',title='Success vs Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version',title=f"Success vs Payload mass for site: {entered_site}")
        return  fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jun/2023 15:40:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:26] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:26] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:26] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:26] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2023 15:40:41] "POST /_dash-